# Job Data Analysis and Salary Prediction

In this notebook, I will analyze the job data I scraped from Indeed.com for data scientist position from my list of cities. While most listings do not come with salary information, being to able extrapolate and predict the expected salaries from other listings can help guide negotiations or at least an insight of what to expect if you are new to the job market like me. 

Normally, regression could be used for a task like this; however, since there is a fair amount of natural variance in job salaries, I approached this as a classification problem using random forest classifier.

### Overview of data :
* There are 630,988 results found for Data Scientist from my list of 86 cities. 
* 590,466 jobs (93.58%) with no salary
* 615,441 (97.54%) of the entries are duplicated
* The salaries are given as text and usually with ranges

### Results:
The accuracy score and across validation score of each Random Forest models shown below:

|Features|Accuracy|Cross Validation|
|------|------|------|
|City|0.655|0.538 ± 0.108|
|Summary|0.704|0.731 ± 0.065|
|Title|0.805|0.804 ± 0.057|
|All 3 above|0.81|0.77 ± 0.073|

In [2]:
#Import packages
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## Explore the Scraped Data 

In [14]:
results = pd.read_csv('./data/Indeed_not_cleaned_long.csv')
print('Total of %s jobs found for Data Scientist'%(results.shape[0]))

Total of 630988 jobs found for Data Scientist


In [15]:
results.nunique()

Unnamed: 0    630988
city_key          86
Title           9771
Company         4103
Location        1581
Summary        12840
Salary           494
dtype: int64

In [43]:
results = results[['Title', 'Company', 'Location', 'Summary','Salary']]
results.head()

,Title,Company,Location,Summary,Salary
0,Associate Data Scientist,DISNEY,NaN,"In this role, you ...",NaN
1,Development and Application Scientist,FluxErgy,NaN,Analyze data gener...,NaN
2,"Data Scientist, Marketing - Fast Growth Ecommerce",Harnham,NaN,"Data Scientist, Marketing - Fast Growth Ecomme...","$12,000 - $13,000 a year"
3,Senior Data Scientist,Cox Automotive,NaN,The Senior Data Sc...,NaN
4,Product Content Strategist,Joybird,NaN,Contribute to a pr...,NaN


In [25]:
#Percentage of missing data.
def missing_data(df):
    no_null = df.notnull().sum()
    missing = df.isnull().sum()
    percent = (df.isnull().sum()/df.isnull().count()*100)
    missing_data = pd.concat([no_null,missing, percent], axis=1, 
                             keys=['NO-null','Missing', 'Percent Missing'])
    return missing_data.sort_values(by='Missing', ascending=False)
missing_data(results)

,NO-null,Missing,Percent Missing
Salary,40522,590466,93.578008
Location,423307,207681,32.913621
Company,630982,6,0.000951
Title,630988,0,0.000000
Summary,630988,0,0.000000


In [26]:
print('Total number of jobs %s'%(results.shape[0]))
print('Number of duplicated results: ',results.duplicated().sum(),'\n')
print('Total unique job titles: ',results.Title.nunique())
print('Total unique companies: ',results.Company.dropna(how='all').nunique())
print('Total unique locations: ',results.Location.dropna(how='all').nunique())

Total number of jobs 630988
Number of duplicated results:  615441 

Total unique job titles:  9771
Total unique companies:  4103
Total unique locations:  1581


### Overview of data :
* There are 630,988 results found for Data Scientist from my list of 86 cities. 
* 590,466 jobs (93.58%) with no salary)
* 615,441 (97.54%) of the entries are duplicated
* The salaries are given as text and usually with ranges
* Some of the salaries are not yearly but hourly or weekly, these will not be useful for now

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). 

In [57]:
salaries = results[results.Salary.notnull()].drop_duplicates()

print('Entries with salary: ',salaries.shape)
salaries.head()

Entries with salary:  (946, 5)


,Title,Company,Location,Summary,Salary
2,"Data Scientist, Marketing - Fast Growth Ecommerce",Harnham,NaN,"Data Scientist, Marketing - Fast Growth Ecomme...","$12,000 - $13,000 a year"
15,"Data Scientist, Marketing - Ecommerce",Harnham,NaN,"Data Scientist, Marketing - Ecommerce. As a Da...","$110,000 - $120,000 a year"
51,Scientist,Alternative Biomedical Solutions,"Irvine, CA",Data management reporting. Alternative BioMedi...,$65 - $75 an hour
63,Business Analyst,上海冰鉴信息科技有限公司IceKredit,"Los Angeles, CA","Partner up with data scientists, a...","$60,000 - $80,000 a year"
64,Administrative Support Coordinator II,Cal State Fullerton,"Fullerton, CA",Provides coordination between the ...,"$3,115 - $5,475 a month"


In [58]:
salaries = salaries[(~salaries.Salary.str.contains('an hour')) & (~salaries.Salary.str.contains('a month'))
                   & (~salaries.Salary.str.contains('a week')) & (~salaries.Salary.str.contains('a day'))]
salaries.shape


(751, 5)

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [59]:
#salaries.salary = salaries.Salary.str.replace('a year', '').str.replace(',', '').str.replace('$', '')

def salary_strip(dataframe, column):
    # remove \$ sign
    dataframe[str(column)] = dataframe[str(column)].replace({'$':''}, regex = True)
    # remove \D
    dataframe[str(column)].replace(regex=True,inplace=True,to_replace=r'\D',value=r' ')
    # remove blank space
    dataframe[str(column)] = dataframe[str(column)].str.replace(' ',',')
    
    dataframe = dataframe.join(dataframe[str(column)].str.split(',,,', 1, expand=True).rename(columns={0:'Low', 1:'High'}))
    # add value to Low column without comma and convert to float 
    dataframe['Low'] = dataframe['Low'].str.replace(',','')
    dataframe['Low'] = dataframe['Low'].astype('float64')
    dataframe.drop(str(column), axis=1, inplace=True)
    
    dataframe['High'] = dataframe['High'].str.replace(',','')
    dataframe['High'] = dataframe['High'].apply(pd.to_numeric)
    # Calculate Average 
    dataframe['Average'] = dataframe[['Low', 'High']].mean(axis=1)
    return dataframe

In [60]:
jobs1 = salary_strip(salaries, 'Salary')
jobs1.tail()


,Title,Company,Location,Summary,Low,High,Average
611624,Research Plant Physiologist,US Department of Agriculture,"Davis, CA 95616 (Central Davis area)",The incumbent would have the oppor...,66253.0,86132.0,76192.5
611631,Junior Specialist – Postharvest Biology and Mi...,"University of California, Davis","Davis, CA 95616 (Central Davis area)","Summarize data, perform data analy...",39500.0,NaN,39500.0
611635,Bioanalytical Manager,Hygieia Biological Laboratories,"Woodland, CA",Data analysis and report preparation. Ability ...,100000.0,200000.0,150000.0
611648,Research Microbiologist/Research Geneticist,US Department of Agriculture,"Davis, CA 95616 (Central Davis area)",Two years of data pertaining to th...,66253.0,86132.0,76192.5
611687,Bioanalytical Manager,Hygieia Biological Laboratories,NaN,This position has ...,100000.0,200000.0,150000.0


In [61]:
def city_strip(dataframe, column):
    dataframe = dataframe.join(dataframe[str(column)].str.split(',', 1, expand=True).rename(columns={0:'City', 1:'State'}))
    return dataframe

In [62]:
job1_city = city_strip(jobs1,'Location')
job1_city.tail(3)

,Title,Company,Location,Summary,Low,High,Average,City,State
611635,Bioanalytical Manager,Hygieia Biological Laboratories,"Woodland, CA",Data analysis and report preparation. Ability ...,100000.0,200000.0,150000.0,Woodland,CA
611648,Research Microbiologist/Research Geneticist,US Department of Agriculture,"Davis, CA 95616 (Central Davis area)",Two years of data pertaining to th...,66253.0,86132.0,76192.5,Davis,CA 95616 (Central Davis area)
611687,Bioanalytical Manager,Hygieia Biological Laboratories,NaN,This position has ...,100000.0,200000.0,150000.0,NaN,NaN


In [66]:
print('Values in State columns contain zipcodes and other information:\n')
print(job1_city.State.unique())

Values in State columns contain zipcodes and other information:

[nan ' CA' ' CA 92660' ' CA 92626' ' CA 91801' ' CA 90245' ' CA 91739'
 ' CO 80223 (Southwestern Denver area)' ' CO' ' PA 17402' ' CA 92656'
 ' CA 90211' ' CA 90046' ' PA' ' PA 19103 (Belmont area)' ' NJ'
 ' PA 19355' ' DE' ' PA 19426' ' FL' ' MN 55441' ' MN' ' TX'
 ' TX 77030 (Medical area)' ' TX 77072 (Sugarland area)'
 ' TX 77007 (Rice Military area)' ' TX 77056 (Galleria-Uptown area)'
 ' TX 77074 (Bellaire area)' ' TX 77001' ' TX 77077 (West Houston area)'
 ' TN' ' CA 94129' ' CA 94598' ' CA 94107 (South Of Market area)'
 ' CA 94109 (Nob Hill area)' ' OR' ' DC' ' MD' ' VA 22102'
 ' DC 20036 (Downtown area)' ' VA' ' MD 20814' ' MD 20878' ' AZ'
 ' WA 98121 (Belltown area)' ' WA' ' WA 98007 (Lake Hills area)' ' IL'
 ' IL 60603 (Loop area)' ' MO' ' MO 63143' ' MO 63011' ' NY'
 ' NY 10018 (Clinton area)' ' NY 10032 (Washington Heights area)'
 ' NY 10010 (Gramercy area)' ' NY 10014 (West Village area)'
 ' NY 10016 (Gramercy

Because of th messy information for State, which we will need later, I will clean this and extract only state initials. 

In [69]:
job1_city['State_initial'] = job1_city['State'][job1_city['State'].notnull()].astype(str).str[0:3]

print('States with only initials after cleaning:\n')
print(job1_city.State_initial.unique())


States with only initials after cleaning:

[nan ' CA' ' CO' ' PA' ' NJ' ' DE' ' FL' ' MN' ' TX' ' TN' ' OR' ' DC'
 ' MD' ' VA' ' AZ' ' WA' ' IL' ' MO' ' NY' ' GA' ' OH' ' NC' ' KS' ' MA'
 ' IN' ' KY']


In [71]:
# Save file before prediction steps
job1_city.to_csv('Indeed_cleaned.csv', encoding='utf-8')

## Predicting Salaries using Random Forests

We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [73]:
df_more = job1_city 
median = df_more['Average'].median()
print ('The median salary for our data set is $' + str(median))

The median salary for our data set is $92404.0


In [83]:
def above_median(x):
    if x > median:
        return 1
    return 0

df_more['Above Median'] = df_more['Average'].apply(above_median)
print('There are %s jobs with salary above median'%(df_more['Above Median'][df_more['Above Median'] == 1].shape[0]))
df_more.head()

There are 374 jobs with salary above median


,Title,Company,Location,Summary,Low,High,Average,City,State,State_initial,Above Median
2,"Data Scientist, Marketing - Fast Growth Ecommerce",Harnham,NaN,"Data Scientist, Marketing - Fast Growth Ecomme...",12000.0,13000.0,12500.0,NaN,NaN,NaN,0
15,"Data Scientist, Marketing - Ecommerce",Harnham,NaN,"Data Scientist, Marketing - Ecommerce. As a Da...",110000.0,120000.0,115000.0,NaN,NaN,NaN,1
63,Business Analyst,上海冰鉴信息科技有限公司IceKredit,"Los Angeles, CA","Partner up with data scientists, a...",60000.0,80000.0,70000.0,Los Angeles,CA,CA,0
74,"Data Scientist, Marketing - Ecommerce",Harnham,NaN,As a Data Scientis...,110000.0,120000.0,115000.0,NaN,NaN,NaN,1
96,"Scientist I,II and III",USA Staffing Services,"Corona, CA","Assists with QC's data management,...",45000.0,65000.0,55000.0,Corona,CA,CA,0


What is the baseline accuracy for this model?

In [80]:
df_more['Above Median'].value_counts()

0    377
1    374
Name: Above Median, dtype: int64

Baseline accuracy for our model would be about 50/50 because if we flipped a coin to determine whether or not a value is above or below the median, we have a 50/50 chance.



In [97]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from ipywidgets import *
from IPython.display import display
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.cross_validation import train_test_split

import warnings
warnings.filterwarnings("ignore")


### City Random Forest

In [98]:
df_city_dummy = pd.get_dummies(df_more['City'])
df_state_dummy = pd.get_dummies(df_more['State_initial'])

X_city = df_city_dummy
y_city = df_more['Above Median']

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X_city, y_city, test_size=0.3, random_state=90)

In [101]:
rfc = RandomForestClassifier(n_estimators=300, random_state=90)
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)
acc = accuracy_score(y_test, rfc_pred)
print("Accuracy Score:", acc.round(3))

s = cross_val_score(rfc, X_city, y_city, cv=10, n_jobs=-1)
print("Cross Validation Score:\t{:0.3} ± {:0.3}".format(s.mean().round(3), s.std().round(3)))


Accuracy Score: 0.655
Cross Validation Score:	0.538 ± 0.108


In [107]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X_city.columns).reset_index()
feature_importances.columns = ['feature', 'importance']

feature_medians = []
for i in X_city.columns:
    feature_medians.append(np.median(df_more[df_more.City == i].Average))

feature_importances['median_salary'] = feature_medians
feature_importances['over_or_under'] = [1 if i > median else 0 for i in feature_importances.median_salary]

feature_importances.sort_values('importance', ascending=False).head(15)

,feature,importance,median_salary,over_or_under
104,Queens,0.068905,78790.5,0
116,San Francisco,0.049749,157500.0,1
76,Manhattan,0.045637,77321.5,0
87,New York,0.036193,120000.0,1
129,St. Louis,0.028170,54974.5,0
27,Chicago,0.026528,115000.0,1
95,Parlier,0.020676,70402.0,0
48,Fort Meade,0.020379,78102.0,0
91,Oakland,0.018616,107500.0,1
78,Menlo Park,0.013892,95000.0,1


### Summary (Job Description) Count Vectorizer

In [108]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer


In [112]:
salaries_w_desc = df_more[df_more.Summary.notnull()]

X_summ = salaries_w_desc.Summary
y_summ = salaries_w_desc['Above Median']

In [113]:
cv = CountVectorizer(stop_words="english")
cv.fit(X_summ)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [114]:
len(cv.get_feature_names())

2352

In [115]:
X_summ_trans = pd.DataFrame(cv.transform(X_summ).todense(), columns=cv.get_feature_names())


In [116]:
X_train, X_test, y_train, y_test = train_test_split(np.asmatrix(X_summ_trans), y_summ, test_size=0.3,
                                                    random_state=59, stratify=y_summ)

In [117]:
word_counts = X_summ_trans.sum(axis=0)
word_counts.sort_values(ascending = False).head(20)

data           1129
scientist       346
scientists      158
experience      128
analysis        126
research        123
team            113
senior           69
learning         63
years            58
work             58
analytics        52
science          51
analyze          51
level            49
including        44
machine          44
seeking          42
looking          42
statistical      41
dtype: int64

In [118]:
word_counts.to_csv('indeed-words.csv', encoding='utf-8')

In [120]:
rfc = RandomForestClassifier(200, random_state=59)
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)
acc = accuracy_score(y_test, rfc_pred)
print("Accuracy Score:", acc.round(3))

s = cross_val_score(rfc, X_summ_trans.as_matrix(), y_summ.as_matrix(), cv=10, n_jobs=-1)
print("Cross Validation Score: {:0.3} ± {:0.3}".format(s.mean().round(3), s.std().round(3)))


Accuracy Score: 0.704
Cross Validation Score: 0.731 ± 0.065


In [124]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X_summ_trans.columns).reset_index()
feature_importances.columns = ['feature', 'importance']

feature_medians = []
feature_means = []
for i in X_summ_trans.columns:
    feature_medians.append(np.median(salaries_w_desc[salaries_w_desc.Summary.str.lower().str.contains(i)].Average))
    feature_means.append(np.mean(salaries_w_desc[salaries_w_desc.Summary.str.lower().str.contains(i)].Average))


feature_importances['median_salary'] = feature_medians
feature_importances['mean_salary'] = feature_means
feature_importances['over_or_under'] = [1 if i > median else 0 for i in feature_importances.median_salary]

feature_importances.sort_values('importance', ascending=False).head(20)

,feature,importance,median_salary,mean_salary,over_or_under
573,data,0.028906,97032.00,104732.723868,1
1923,scientist,0.020169,103902.50,109650.642857,1
1411,models,0.014279,136250.00,138717.050000,1
142,analytics,0.013586,120000.00,116960.085106,1
1920,science,0.012760,121250.00,122511.460000,1
1297,machine,0.011670,130000.00,132858.320513,1
1239,learning,0.011028,125000.00,124425.453704,1
1953,senior,0.010514,147500.00,144535.719298,1
135,analysis,0.009478,79249.50,95580.056911,0
1219,laboratory,0.008884,65046.00,66267.089286,0


### Title Count Vectorizer

In [125]:
salaries_w_desc = df_more[df_more.Summary.notnull()]

X_title  = salaries_w_desc.Title
y_title = salaries_w_desc['Above Median']


In [126]:
cv = CountVectorizer(stop_words="english")
cv.fit(X_title)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [127]:
X_title_trans = pd.DataFrame(cv.transform(X_title).todense(), columns=cv.get_feature_names())


In [128]:
X_train, X_test, y_train, y_test = train_test_split(X_title_trans, y_title, test_size=0.3, random_state=59)


In [130]:
rfc = RandomForestClassifier(200, random_state=59)
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)
acc = accuracy_score(y_test, rfc_pred)
print("Accuracy Score:", acc.round(3))

s = cross_val_score(rfc, X_title_trans.as_matrix(), y_title.as_matrix(), cv=10, n_jobs=-1)
print("Cross Validation Score: {:0.3} ± {:0.3}".format(s.mean().round(3), s.std().round(3)))


Accuracy Score: 0.805
Cross Validation Score: 0.804 ± 0.057


In [131]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X_title_trans.columns).reset_index()
feature_importances.columns = ['feature', 'importance']

feature_medians = []
feature_means = []
for i in X_title_trans.columns:
    feature_medians.append(np.median(salaries_w_desc[salaries_w_desc.Title.str.lower().str.contains(i)].Average))
    feature_means.append(np.mean(salaries_w_desc[salaries_w_desc.Title.str.lower().str.contains(i)].Average))


feature_importances['median_salary'] = feature_medians
feature_importances['mean_salary'] = feature_means
feature_importances['over_or_under'] = [1 if i > median else 0 for i in feature_importances.median_salary]

feature_importances.sort_values('importance', ascending=False).head(20)

,feature,importance,median_salary,mean_salary,over_or_under
156,data,0.120067,122500.00,126239.930233,1
508,scientist,0.039114,110000.00,112825.797710,1
491,research,0.032214,71351.00,78849.897059,0
514,senior,0.021638,127500.00,127548.668421,1
208,environmental,0.018186,60000.00,65146.860000,0
147,coordinator,0.017801,62693.00,62906.942308,0
93,bureau,0.017094,78057.50,78431.236842,0
203,engineer,0.016172,105000.00,114483.043860,1
48,analyst,0.016027,79249.50,83588.129630,0
536,sr,0.014207,142500.00,135369.444444,1


### Combining Title CV, Summary CV, and Location

In [132]:
salaries_w_desc = df_more[df_more.Summary.notnull()].reset_index()
city_dummies = pd.get_dummies(salaries_w_desc.City)

X = pd.concat([city_dummies, X_title_trans, X_summ_trans], axis=1)
y = salaries_w_desc['Above Median']

In [134]:
print(X.shape)
print(y.shape)

(751, 3114)
(751,)


In [135]:
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y, test_size=0.3, random_state=68, stratify=y)


In [136]:
rfc = RandomForestClassifier(500, random_state=59)
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)
acc = accuracy_score(y_test, rfc_pred)
print("Accuracy Score:", acc.round(3))

s = cross_val_score(rfc, X.as_matrix(), y.as_matrix(), cv=10, n_jobs=-1)
print("Cross Validation Score: {:0.3} ± {:0.3}".format(s.mean().round(3), s.std().round(3)))

Accuracy Score: 0.81
Cross Validation Score: 0.77 ± 0.073


In [137]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X.columns).reset_index()
feature_importances.columns = ['feature', 'importance']

feature_medians = []
for i in city_dummies.columns:
    feature_medians.append(np.median(df_more[df_more.City == i].Average))
for i in X_title_trans.columns:
    feature_medians.append(np.median(salaries_w_desc[salaries_w_desc.Title.str.lower().str.contains(i)].Average))
for i in X_summ_trans.columns:
    feature_medians.append(np.median(salaries_w_desc[salaries_w_desc.Summary.str.lower().str.contains(i)].Average))

feature_importances['median_salary'] = feature_medians
feature_importances['over_or_under'] = [1 if i > median else 0 for i in feature_importances.median_salary]

feature_importances.sort_values('importance', ascending=False).head(20)

,feature,importance,median_salary,over_or_under
301,data,0.043239,122500.0,1
653,scientist,0.013876,110000.0,1
1335,data,0.013451,97032.0,1
2685,scientist,0.013178,103902.5,1
318,director,0.010993,110511.5,1
636,research,0.010826,71351.0,0
193,analyst,0.007565,79249.5,0
678,specialist,0.006916,65000.0,0
348,engineer,0.006800,105000.0,1
2059,machine,0.006525,130000.0,1
